In [ ]:
%pip install openai pandas

In [2]:
from openai import OpenAI
import pandas as pd

In [3]:
# Funções para estrair dados de um arquivo CSV

def resumo_dados(df):
    return df.describe(include='all').to_string()


def tabela_completa_str(df, max_linhas=100):
    # Verifica se o DataFrame está vazio
    if df.empty:
        return "Nenhum dado disponível."
    
    # Verifica o número total de linhas
    linhas_totais = len(df)

    # Se o número de linhas for maior que o máximo permitido, exibe apenas uma parte
    if linhas_totais > max_linhas:
        top = df.head(max_linhas // 2).to_string(index=False)
        bottom = df.tail(max_linhas // 2).to_string(index=False)
        return f"{top}\n...\n{bottom}\n[Exibindo {max_linhas} de {linhas_totais} linhas]"
    
    # Caso contrário, exibe o DataFrame completo
    else:
        return df.to_string(index=False)

In [4]:
# Carregando o arquivo CSV
df = pd.read_csv('planilha_completa.csv', encoding='latin1')
resumo = resumo_dados(df)
tabela_completa = tabela_completa_str(df)

In [5]:
client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key="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",
)

In [6]:
# Prompt inicial para o TUPÃ.
prompt = f"""
Você é TUPÃ, uma inteligência capaz de analisar dados de planilhas.
Aqui estão os dados que você deve considerar:
{tabela_completa}

Aqui estão os dados resumidos:
{resumo}

Com base nesses dados, responda à pergunta do usuário de forma objetiva e educada.
"""

In [7]:
# Iniciando o TUPÃ.
print("\nTUPÃ iniciado. Digite 'sair' para encerrar.\n")

# Loop principal para interação com o usuário
while True:
    user_input = input("Usuario: ")
    if user_input.lower() == 'sair':
        break

    # Tentando obter resposta da API
    try:
        resposta_api = client.chat.completions.create(
            model="deepseek/deepseek-chat-v3-0324:free", # Modelo a ser usado (ajustável conforme necessário)
            messages = [
                {"role": "system", "content": prompt},
                {"role": "user", "content": user_input}
            ],
            max_tokens=700 # Limite de tokens para a resposta (ajustável conforme necessário)
        )

        print(f"Resposta da API: {resposta_api}")
        resposta_texto = resposta_api.choices[0].message.content

    except Exception as e:
        print(f"Erro na API: {e}")

    # Exibindo a resposta do TUPÃ
    print("\nUsuario: ", user_input)
    print(f"\nTUPÃ: {resposta_texto}")


TUPÃ iniciado. Digite 'sair' para encerrar.

Resposta da API: ChatCompletion(id='gen-1753018060-G4yxG6DqYN1ZBIDlsug4', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Aqui estão as informações mais relevantes da planilha:\n\n1. **Cobertura Temporal**:  \n   - Os dados abrangem os meses de **janeiro a junho** (JAN a JUN), com valores zerados de **julho a dezembro** (JUL a DEZ), indicando que o acompanhamento foi realizado apenas no primeiro semestre.  \n\n2. **Total Geral**:  \n   - **Realizado**: 9.559  \n   - **Programado**: 14.152  \n   - **Execução Anual**: 68% ("Merece Atenção").  \n\n3. **Situações Destacadas**:  \n   - **Acima do Programado** (3 unidades):  \n     - Marabá (111%), CCBS-Belém (123%), Planetário-Belém (1.700%).  \n   - **Abaixo do Programado** (3 unidades):  \n     - Educação Física-Belém (31%), São Miguel do Guamá (42%), Salvaterra (37%).  \n   - **Conforme Programado** (2 unidades):  \n     - Santarém (89%), B

KeyboardInterrupt: 